In [12]:
from pyelasticsearch.client import ElasticSearch
import json
import requests
import pandas as pd
import math
import numpy as np
import boto3

In [13]:
config = json.load(open('../config/configsql.json', 'r'))
aws_access_key_id=config['dev']['aws']['key']
aws_secret_access_key=config['dev']['aws']['secret']
region = 'us-east-1'
endpoint = config['dev']['endpoint']
bucketname=config['dev']['bucketname']['bucketname']
s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
index_name1 = 'candidates_cv'
index_name2 = 'parsed_resume'
es = ElasticSearch(endpoint, port=443)

# all dict

In [14]:
d_all = es.search(index_name1, query={
                                     "query": {"match_all": {}},
                                     "size":  622
                                     })

TypeError: search() got multiple values for argument 'query'

In [2045]:
all_user = {}
for i in range(len(d_all['hits']['hits'])):
    user = d_all['hits']['hits'][i]['_source']
    id = d_all['hits']['hits'][i]['_id']
    all_user[id] = user

# parsed resume

In [2050]:
def extract_information(user_record):
    resp = {}
    for element in ['education', 'summary', 'projects', 'experience']:
        if element in user_record:
            text = user_record[element]
            #print(element,len(text))
            entity = comprehend.detect_entities(Text=text, LanguageCode='en')
            keyword = comprehend.detect_key_phrases(Text=text, LanguageCode='en')
        else:
            entity = {}
            keyword = {}
        resp[element] = {"entities": entity, "keywords": keyword}
    return resp

In [2051]:
def ASCII(s):
    if s is None:
        return ''
    s2 = ''
    for i in range(len(s)):
        x = ord(s[i])
        if x >= 32 and x < 128:
            s2 += s[i]
        else:
            s2 += ' '
    return s2

In [2052]:
resp_all = {}
for id in all_user.keys():
    print(id)
    user = all_user[id]
    for key in user.keys():
        user[key] = ASCII(user[key])
        if len(user[key]) > 5000:
            user[key] = user[key][0:5000]
    result = extract_information(user)
    resp_all[id] = result

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_B. TolgaOztanResume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_MarkoBokulic_resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_CV.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Prasuna_Duggirela_Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Karan Kapoor - Data Science - Carnegie Mellon University.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Data Science, Inc..pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Bo_Peng.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Srikar_G.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_resume_minshan.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Zhonglin-jobs-resume_new.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume DA 2.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Liang_Gao_Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_PrauseResume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Shahram Farhadi Nia Resume_2016.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume_anurag_panwar_pub.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/t

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume of Ramakrishnan, Vijay Thyagarajan_Analyst.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_RESUME-CHIRAG_SUBRAMANIAN_CHARLES_RIVER.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume_Qin_Wen.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_LONGPEI_RESUME_UPDATED.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Ankit_resume-FA (1).pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Nikita_Kirnosov_resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume_Adharsh Rajendran_31Dec.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_cd6a7671.pdf.txt
https

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Curriculum_vitae.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume_Michal_Ben_Neria.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Fu_CV.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_ResumeZiboWang.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_RESUME.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Gowdhaman_Sadhasivam_RESUME.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_9e94608b.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_9f088375.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_8d15b6c3.pd

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Shashank Sharma.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_RuslanaDalinina.Kinetic.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume_Samet_Keserci_Sept29.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_CV_RaptisK-2.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_ResumeSubramanianVellaiyan.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume-SanginKim.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume_Yilun_Wang2.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Kevin_Tran.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unso

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_7cff1979.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Adam Velez Resume .pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_965a2b8d.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_ResumeMohammadNiknazar.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_6084aa80.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_48dcbbee.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_41a0dbe2.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_38375814.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_51b75f0d.pdf.txt
https://s3.am

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_QianYao-Data-Science-Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Maxim.Arap.Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume JP.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume - Plotner_Rhett.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Amith_Mandula_Resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Resume copy.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Daniel_Kim_1757332.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Peter Park - Resume - Jan 2016.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resum

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_1ee4a9f0.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_2016 AllenKatz_CV Data Scientist.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_249d5333.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_28b2c4c1.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_2fab213d.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_resume_full_version2.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_b6f2c5b9.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Anup_Kalia.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_ResumeMadihaHuss

https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_GeorginaTolgosResume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Daniel_Kim_1757332(1).pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_XSS_resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Tejapr6.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_resume_Le.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_patri_resume1pg.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_kts_resume.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_resume (1) copy.pdf.txt
https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_krivanw.pdf

In [2055]:
len(resp_all)

622

# update to elasticsearch

In [2056]:
for id in resp_all.keys():
    user = resp_all[id]
    result = es.index(index_name2,'parsed_resume',user, id = id )

In [2063]:
top10 = es.search(index = index_name2, query={"query": {"match_all": {}}})

In [2095]:
top10['hits']['hits'][1]['_source']['education']['keywords']['KeyPhrases'][1]['Text'].lower()

'computing'

In [2163]:
top10['hits']['hits'][1]['_source']['education']['keywords']['KeyPhrases']

[{'BeginOffset': 0,
  'EndOffset': 8,
  'Score': 0.9637526273727417,
  'Text': 'Bachlors'},
 {'BeginOffset': 12,
  'EndOffset': 21,
  'Score': 0.9738655090332031,
  'Text': 'Computing'},
 {'BeginOffset': 23,
  'EndOffset': 30,
  'Score': 0.9676105976104736,
  'Text': 'Honours'},
 {'BeginOffset': 33,
  'EndOffset': 46,
  'Score': 0.9351752996444702,
  'Text': 'Expected 2016'},
 {'BeginOffset': 51,
  'EndOffset': 56,
  'Score': 0.984465479850769,
  'Text': 'Queen'},
 {'BeginOffset': 59,
  'EndOffset': 69,
  'Score': 0.9880484342575073,
  'Text': 'University'},
 {'BeginOffset': 73,
  'EndOffset': 79,
  'Score': 0.9994682669639587,
  'Text': 'Canada'}]

In [15]:
es.search(index = index_name2, query=
{'_source':'education.keywords.KeyPhrases.Text',
  "aggs": {
    "group_by_keyword": {
      "terms": {
        "field": "education.keywords.KeyPhrases.Text.keyword"
      }
    }
  }
})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'aggregations': {'group_by_keyword': {'buckets': [{'doc_count': 198,
     'key': 'University'},
    {'doc_count': 170, 'key': 'Science'},
    {'doc_count': 86, 'key': 'GPA'},
    {'doc_count': 74, 'key': 'Machine Learning'},
    {'doc_count': 67, 'key': 'Computer Science'},
    {'doc_count': 57, 'key': 'Technology'},
    {'doc_count': 55, 'key': 'data'},
    {'doc_count': 52, 'key': 'Python'},
    {'doc_count': 50, 'key': 'Master'},
    {'doc_count': 48, 'key': 'Engineering'}],
   'doc_count_error_upper_bound': 32,
   'sum_other_doc_count': 24727}},
 'hits': {'hits': [{'_id': 'https://s3.amazonaws.com/careers.dataskeptic.com/resumes/unsourced/text/resumes_unsourced_Updated_Resume_Ayan.pdf.txt',
    '_index': 'parsed_resume',
    '_score': 1.0,
    '_source': {'education': {'keywords': {'KeyPhrases': [{'Text': 'University'},
        {'Text': 'Washington'},
        {'Text': 'Information School'},
        {'Text': 'Seattle'},
      

In [2453]:
count100 = es.search(index = index_name2, query=
    {
      "_source": 'education.keywords.KeyPhrases.Text',
      "size": 0,
      "aggs": {
        "group_by_coll_conv": {
          "terms": {
            "field": "education.keywords.KeyPhrases.Text.keyword",
            "size" : 500
          },
          "aggs": {
            "count" : {
              "value_count" : 
                { 
                  "field" : "education.keywords.KeyPhrases.Text.keyword" 
                } 
            }
        }
    }
}})

In [2454]:
count100

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'aggregations': {'group_by_coll_conv': {'buckets': [{'count': {'value': 11712},
     'doc_count': 198,
     'key': 'University'},
    {'count': {'value': 7781}, 'doc_count': 170, 'key': 'Science'},
    {'count': {'value': 4486}, 'doc_count': 86, 'key': 'GPA'},
    {'count': {'value': 5846}, 'doc_count': 74, 'key': 'Machine Learning'},
    {'count': {'value': 3077}, 'doc_count': 67, 'key': 'Computer Science'},
    {'count': {'value': 6111}, 'doc_count': 58, 'key': 'Python'},
    {'count': {'value': 2965}, 'doc_count': 57, 'key': 'Technology'},
    {'count': {'value': 6208}, 'doc_count': 55, 'key': 'data'},
    {'count': {'value': 1843}, 'doc_count': 50, 'key': 'Master'},
    {'count': {'value': 3711}, 'doc_count': 48, 'key': 'Engineering'},
    {'count': {'value': 3271}, 'doc_count': 46, 'key': 'Data Mining'},
    {'count': {'value': 3411}, 'doc_count': 46, 'key': 'Los Angeles'},
    {'count': {'value': 2524}, 'doc_count': 41, 'ke

In [2298]:
l_100 = count100['aggregations']['group_by_coll_conv']['buckets']

In [2299]:
keys = []
for i in l_100:
    key = i['key']
    keys.append(key)

In [2300]:
doc_counts = []
for i in l_100:
    doc_count = i['doc_count']
    doc_counts.append(doc_count)

In [2301]:
values = []
for i in l_100:
    value = i['count']['value']
    values.append(value)

In [2302]:
df = pd.DataFrame(
    {'word': keys,
     'doc_count': doc_counts,
     'values': values
    })

In [2303]:
count100['hits']['total']

622

In [2304]:
df['idf'] = np.log(count100['hits']['total']/df['doc_count'])

In [2305]:
df['tf'] = df['values']/sum(df['values'])

In [2306]:
df['tf-idf'] = df['idf']*df['tf']
df.head()

,doc_count,values,word,idf,tf,tf-idf
0,198,11712,University,1.144673,0.031296,0.035824
1,170,7781,Science,1.297142,0.020792,0.026970
2,86,4486,GPA,1.978593,0.011987,0.023718
3,74,5846,Machine Learning,2.128875,0.015621,0.033256
4,67,3077,Computer Science,2.228247,0.008222,0.018321


In [2495]:
pd.options.display.max_rows = 9999

In [2312]:
df_s = df.sort_values(by=['tf','word'], ascending=[False, True])

# all source

In [17]:
sources = ['education', 'summary','experience','projects']

In [18]:
results = []
for element in sources:
    result = es.search(index = index_name2, query=
    {
      "_source": element + '.keywords.KeyPhrases.Text',
      "size": 0,
      "aggs": {
        "group_by_coll_conv": {
          "terms": {
            "field": element + ".keywords.KeyPhrases.Text.keyword",
            "size" : 500
          },
          "aggs": {
            "count" : {
              "value_count" : 
                { 
                  "field" : element + ".keywords.KeyPhrases.Text.keyword"
                } 
            }
        }
    }
}})
    results.append(result)

In [19]:
results_all = []
for i in range(len(results)):
    result_all = results[i]['aggregations']['group_by_coll_conv']['buckets']
    results_all.append(result_all)

In [ ]:
dfs = []
for j in range(len(sources)):
    source = sources[j]
    d = results_all[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': sources
        })
    dfs.append(aa)

df = pd.concat(dfs)

In [21]:
dfs = []
for j in range(len(sources)):
    source = sources[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': source
        })
    dfs.append(aa)

df = pd.concat(dfs)
df.reset_index(drop=True)

,doc_count,source,values,word
0,198,education,11712,University
1,170,education,7781,Science
2,86,education,4486,GPA
3,74,education,5846,Machine Learning
4,67,education,3077,Computer Science
5,58,education,6111,Python
6,57,education,2965,Technology
7,55,education,6208,data
8,50,education,1843,Master
9,48,education,3711,Engineering


In [22]:
results[0]['hits']['total']

622

In [2538]:

dfs = []
for j in range(len(sources)):
    source = sources[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    for i in results_all[j]:
        idfs = np.log([results[0]['hits']['total']/x for x in doc_counts]).tolist()
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': source,
         'idf':idfs
        })
    dfs.append(aa)

df = pd.concat(dfs)
df.reset_index(drop=True)

In [23]:
df['values']/sum(df['values'])
dfs = []
for j in range(len(sources)):
    source = sources[j]
    keys = []
    for i in results_all[j]:
        key = i['key']
        keys.append(key)
    doc_counts = []
    for i in results_all[j]:
        doc_count = i['doc_count']
        doc_counts.append(doc_count)
    values = []
    for i in results_all[j]:
        value = i['count']['value']
        values.append(value)
    for i in results_all[j]:
        idfs = np.log([results[0]['hits']['total']/x for x in doc_counts]).tolist()
    for i in results_all[j]:
        tf = [x/sum(values) for x in values]
    for i in result_all[j]:
        tf_idf = [a*b for a,b in zip(tf,idfs)]
    aa = pd.DataFrame(
        {'word': keys,
         'doc_count': doc_counts,
         'values': values,
         'source': source,
         'idf':idfs,
         'tf':tf,
         'tf_idf':tf_idf
        })
    dfs.append(aa)

df = pd.concat(dfs)
df.reset_index(drop=True,inplace = True)
df= df.sort_values(by=['tf','word'], ascending=[False, True])

In [29]:
df.sort_values('word')

,doc_count,idf,source,tf,tf_idf,values,word
1819,2,5.739793,projects,0.000934,0.005360,200,"""AVR Microcontrollers"
1820,2,5.739793,projects,0.000934,0.005360,200,"""Dynamical Systems"
1821,2,5.739793,projects,0.000934,0.005360,200,"""Linear and Nonlinear Kalman Filter"
1822,2,5.739793,projects,0.000934,0.005360,200,"""Linear and Nonlinear Kalman Filters"
1823,2,5.739793,projects,0.000934,0.005360,200,"""Signal Processing"
1824,2,5.739793,projects,0.000934,0.005360,200,"""Single-channel and Multi-channel Blind"
429,2,5.739793,education,0.001235,0.007086,462,#ED33C-0949
430,2,5.739793,education,0.001235,0.007086,462,#H33G-1475
431,2,5.739793,education,0.001235,0.007086,462,#H34D-03. 2 [13] Sara Sadri
432,2,5.739793,education,0.001235,0.007086,462,#H53J-1547


In [31]:
df2 = df

In [46]:
df2['word'] = [x.lower() for x in df2['word']]

In [52]:
df3 = pd.DataFrame(df2.groupby(['word'])['word'].count())

In [54]:
df3.columns = ['count']

In [56]:
df3 = df3.reset_index()

In [59]:
df3.head(10)

,word,count
0,"""avr microcontrollers",1
1,"""dynamical systems",1
2,"""linear and nonlinear kalman filter",1
3,"""linear and nonlinear kalman filters",1
4,"""signal processing",1
5,"""single-channel and multi-channel blind",1
6,#ed33c-0949,1
7,#h33g-1475,1
8,#h34d-03. 2 [13] sara sadri,1
9,#h53j-1547,1


In [68]:
def num_there(s):
    return any(i.isdigit() for i in s)

In [70]:
word_list = df3['word'].tolist()

In [81]:
df_num = pd.DataFrame([num_there(i) for i in word_list])

In [83]:
df4 = pd.concat([df3,df_num], axis=1)

In [91]:
df_no_num = df4[df4.loc[:][0]==False]

In [141]:
df_no_num .reset_index(drop=True, inplace=True)

In [113]:
# ''.join(filter(str.isalnum, string))

In [112]:
import re
re.sub(r'\W+', ' ', string)

' avr microcontrollers'

In [114]:
def remove_spe_chara(string):

    return re.sub(r'\W+', ' ', string)

In [145]:
no_spec_list = df_no_num['word'].tolist()

In [146]:
df_no_spe = pd.DataFrame([remove_spe_chara(i) for i in no_spec_list])

In [147]:
df_no_spe=df_no_spe.rename(columns = {0:'word2'})

In [166]:
df_final = pd.concat([df_no_num,df_no_spe],axis=1)

In [167]:
df_final=pd.DataFrame(df_final.groupby(['word2'])['word2'].count())

In [172]:
df_final = df_final.reset_index()

In [171]:
df_final.columns = ['count']

In [174]:
df_final.to_clipboard()

In [178]:
df_no_num.head(400)

,word,count,0
0,"""avr microcontrollers",1,False
1,"""dynamical systems",1,False
2,"""linear and nonlinear kalman filter",1,False
3,"""linear and nonlinear kalman filters",1,False
4,"""signal processing",1,False
5,"""single-channel and multi-channel blind",1,False
6,&trainings independent research,1,False
7,' award,2,False
8,' behavior,1,False
9,' competitors,1,False


In [ ]:
# save to parquet
# save to s3



In [2557]:
filename = 'terms.parquet'

In [ ]:
df.to_parquet(filename)

In [2559]:
s3.Bucket(bucketname).upload_file(filename,'sys/parsed-keywords_v1.parquet')